In [4]:
import networkx as nx
import numpy as np
from itertools import permutations, product
from functools import partial
import scipy.sparse as sps
import scipy.linalg as la
from scipy.optimize import minimize, linear_sum_assignment, LinearConstraint, Bounds
import sys
import time
import importlib

sys.path.insert(1, '/home/me/mGH_UB')
importlib.reload(sys.modules['ub'])
from ub import *
from tools import *
from MCMC import sample_points
sys.path.insert(1, '/home/me/persim-fork/persim')
from gromov_hausdorff import estimate, find_lb, find_ub, find_ub_of_min_distortion

In [2]:
rng = np.random.RandomState(0)

n = 100
size = 20
space_type = 'eucl'
D1s = []
D2s = []
for _ in range(n):
    if space_type == 'graph':
        _, D1 = gen_graph(size, .05, enforce_n=False, seed=rng)
    elif space_type == 'eucl':
        D1 = gen_eucl_X(size, rnd=rng)
    else:
        print(f'unknown space type {space_type}')
        break
        
    pi = rng.permutation(len(D1))
    D2 = D1[np.ix_(pi, pi)]
    
    D1s.append(D1)
    D2s.append(D2)
    
diams = [D1.max() for D1 in D1s]
print(f'diam {np.mean(diams):.1f}±{np.std(diams):.1f}')

diam 2.4±0.1


In [12]:
%%time
rng = np.random.RandomState(0)
point_lists = [sample_points(len(D1), len(D2), 10, 1000, rng=rng)
               for D1, D2 in zip(D1s, D2s)]

CPU times: user 1d 4h 53min 58s, sys: 2h 37min 10s, total: 1d 7h 31min 9s
Wall time: 10h 15min 22s


In [68]:
%%time
rng = np.random.RandomState(0)
points = sample_points(size, size, 100, 1000, rng=rng)

CPU times: user 1min 14s, sys: 49.3 s, total: 2min 3s
Wall time: 38.2 s


In [39]:
import sys, importlib
importlib.reload(sys.modules['ub'])
from ub import find_min_dis

In [70]:
%%time
rng = np.random.RandomState(0)

c_seqs = []
for c1 in [1.01, 1.1]:
    for c2 in [None, 2, 5]:
        for c3 in [25, 50]:
            c_seq = [c for c in [c1, c2, c3] if c]
            if c_seq:
                c_seqs.append(c_seq)

# c_seqs = [[1.01, 50]]
for c_seq in c_seqs:
    start = time.time()
    dis = [find_min_dis(D1, D2, c_seq=c_seq, max_iter=25, rng=rng, n_restarts=100)
           for D1, D2 in zip(D1s, D2s)]
    exact_ratio = sum(x == 0 for x in dis) / n
    end = time.time()
    print(f'{n} {space_type} of size {size} {c_seq}: {exact_ratio:.0%} '
          f'{np.mean(dis):.2f}±{np.std(dis):.2f}, {(end-start)/60:.1f}min')

# graph [1.1, 25]: 93% 0.14±0.51, 11.7min (10 restarts at random)

# graph [1.01, 25]: 86% 0.28±0.69, 6.4min (10 restarts at random)
# graph [1.01, 25]: 88% 0.24±0.65, 4.7min (10 restarts using MCMC)

# eucl [1.01, 25]: 0% 3.74±0.15, 8.2min (1 restart from center)
# eucl [1.01, 25]: 0% 3.53±0.10, 79.5min (10 restarts using MCMC)
# eucl [1.01, 25]: 0% 3.56±0.08, 89.6min (10 restarts at random)

100 eucl of size 20 [1.01, 25]: 0% 0.73±0.16, 7.1min
100 eucl of size 20 [1.01, 50]: 2% 0.73±0.19, 7.0min
100 eucl of size 20 [1.01, 2, 25]: 1% 0.69±0.15, 6.7min
100 eucl of size 20 [1.01, 2, 50]: 1% 0.67±0.13, 6.2min
100 eucl of size 20 [1.01, 5, 25]: 0% 0.72±0.15, 6.3min
100 eucl of size 20 [1.01, 5, 50]: 0% 0.68±0.11, 6.1min
100 eucl of size 20 [1.1, 25]: 0% 0.70±0.15, 3.8min
100 eucl of size 20 [1.1, 50]: 0% 0.70±0.15, 3.9min
100 eucl of size 20 [1.1, 2, 25]: 0% 0.72±0.17, 3.9min
100 eucl of size 20 [1.1, 2, 50]: 1% 0.68±0.16, 4.1min
100 eucl of size 20 [1.1, 5, 25]: 1% 0.73±0.16, 4.0min
100 eucl of size 20 [1.1, 5, 50]: 2% 0.68±0.16, 4.0min
CPU times: user 1h 2min 49s, sys: 5.44 s, total: 1h 2min 54s
Wall time: 1h 3min


In [ ]:
'''
eucl [1.01, 25]: 0% 0.73±0.15, 6.1min
eucl [1.01, 50]: 0% 0.74±0.16, 6.3min
eucl [1.01, 2, 25]: 0% 0.70±0.14, 5.8min
eucl [1.01, 2, 50]: 0% 0.71±0.14, 5.9min
eucl [1.01, 5, 25]: 0% 0.71±0.13, 5.8min
eucl [1.01, 5, 50]: 0% 0.71±0.13, 5.8min
eucl [1.1, 25]: 0% 0.71±0.14, 3.8min
eucl [1.1, 50]: 0% 0.71±0.14, 4.0min
eucl [1.1, 2, 25]: 0% 0.71±0.14, 4.2min
eucl [1.1, 2, 50]: 0% 0.71±0.14, 4.4min
eucl [1.1, 5, 25]: 0% 0.71±0.13, 4.2min
eucl [1.1, 5, 50]: 0% 0.71±0.13, 4.1min
'''

In [ ]:
'''
EUCL
[1.01]: 0% 4.04±0.19, 13.1min
[1.1]: 0% 3.89±0.17, 4.7min
[2]: 0% 3.76±0.10, 7.3min
[5]: 0% 3.99±0.26, 16.3min
[25]: 0% 4.04±0.16, 19.7min
[50]: 0% 3.99±0.14, 17.9min

[1.01, 2]: 0% 3.83±0.15, 11.7min
[1.01, 5]: 0% 3.76±0.13, 13.3min
[1.01, 25]: 0% 3.75±0.16, 18.3min
[1.01, 50]: 0% 3.75±0.14, 18.4min
[1.1, 2]: 0% 3.85±0.17, 4.8min
[1.1, 5]: 0% 3.77±0.14, 7.7min
[1.1, 25]: 0% 3.72±0.12, 8.5min
[1.1, 50]: 0% 3.73±0.12, 9.7min
[2, 25]: 0% 3.72±0.09, 8.2min
[2, 50]: 0% 3.70±0.09, 8.3min
[5, 25]: 0% 3.85±0.17, 25.7min
[5, 50]: 0% 3.83±0.16, 29.0min

[1.01, 2, 25]: 0% 3.71±0.10, 14.3min
[1.01, 2, 50]: 0% 3.70±0.12, 15.0min
[1.01, 5, 25]: 0% 3.69±0.11, 15.8min
[1.01, 5, 50]: 0% 3.68±0.10, 14.1min
[1.1, 2, 25]: 0% 3.71±0.12, 8.1min
[1.1, 2, 50]: 0% 3.71±0.12, 8.7min
[1.1, 5, 25]: 0% 3.69±0.12, 8.7min
[1.1, 5, 50]: 0% 3.68±0.11, 8.0min
'''

In [ ]:
'''
GRAPHS
[1.01]: 61% 1.37±1.77, 2.6min
[1.1]: 63% 1.26±1.69, 2.5min
[2]: 28% 2.34±1.54, 2.9min
[5]: 0% 2.77±0.44, 3.9min
[25]: 0% 2.45±0.50, 4.5min
[50]: 0% 2.41±0.51, 4.6min

[1.01, 2]: 61% 1.34±1.73, 2.8min
[1.01, 5]: 61% 1.15±1.46, 3.0min
[1.01, 25]: 61% 0.81±1.03, 3.4min *
[1.01, 50]: 61% 0.81±1.03, 3.6min *
[1.1, 2]: 63% 1.25±1.68, 2.7min
[1.1, 5]: 63% 1.05±1.40, 3.0min
[1.1, 25]: 63% 0.81±1.08, 3.2min *
[1.1, 50]: 63% 0.81±1.08, 3.4min *
[2, 25]: 28% 1.64±1.09, 4.5min
[2, 50]: 28% 1.64±1.09, 4.4min
[5, 25]: 0% 2.49±0.50, 4.2min
[5, 50]: 0% 2.47±0.50, 4.0min

[1.01, 2, 25]: 61% 0.82±1.04, 3.4min
[1.01, 2, 50]: 61% 0.82±1.04, 3.3min
[1.01, 5, 25]: 61% 0.87±1.12, 3.4min
[1.01, 5, 50]: 61% 0.87±1.12, 3.4min
[1.1, 2, 25]: 63% 0.81±1.08, 3.4min
[1.1, 2, 50]: 63% 0.81±1.08, 3.6min
[1.1, 5, 25]: 63% 0.84±1.13, 3.3min
[1.1, 5, 50]: 63% 0.82±1.10, 3.3min
'''

In [3]:
def compile_performance(n, p, max_iter, rel_size_diff, c_seq, show_graph_properties=False):
    perf = performance.get((n, p, max_iter, rel_size_diff, c_seq), None)
    perf_desc = f"{perf['acc']:.1%} correct, {perf['time']:.3f}s" if perf else '-'*10
    if show_graph_properties and perf:
        perf_desc += f" (N={perf['order']}, diam={perf['diam']})"
    if rel_size_diff == 0:
        size = '#X=#Y'
    elif rel_size_diff > 0:
        size = f'#Y/#X={1 + rel_size_diff}'
    else:
        size = f'#X/#Y={1 - rel_size_diff}'
    
    seq = ' → '.join([f'{c}[{n_iter}]' for c, n_iter in zip(c_seq, perf['iters'])])
        
    return (f'n={n} | p={p} | max#i={max_iter} | {size} | c[#i]={seq}: {perf_desc}')
 

In [10]:
#performance = dict()
N = 100 # number of graphs in one dataset

ns = [10, 100, 500, 1000][:4] # [10]
ps = [.25, .05, .01, .005] # [.1]
n_restarts = 16

Ds = {(n, p): [] for n, p in zip(ns, ps)}
P0s_by_n_p = {(n, p): [] for n, p in zip(ns, ps)}
pi_YXs = {(n, p): [] for n, p in zip(ns, ps)}
for n, p in zip(ns, ps):
    for _ in range(N):
        _, D = gen_graph(n, p, enforce_n=False)
        Ds[n, p].append(D)
        P0s_by_n_p[n, p].append([gen_stoch_mx(len(D), len(D)) for _ in range(n_restarts)])
        pi_YXs[n, p].append(np.random.permutation(len(D)))


KeyboardInterrupt: 

In [9]:
verbose = False
i = 0
max_iters = [100]
rel_size_diffs = [0]#[0, .1, .5, 2.5]
c_seqs = [[1.00032, 2], [1.00032, 1.04], [1.0016, 2], [1.0016, 1.2], [1.2], [1.3], [1.4], [1.5], [1.6]]
c_seqs = [[1.00032, 2], [1.0016, 2], [1.008, 2], [1.04, 2], [1.1, 2], [1.2, 2], [1.4, 2], [1.6, 2], [2]]
n_runs = len(ns) * len(max_iters) * len(rel_size_diffs) * len(c_seqs)
for n, p in zip(ns, ps): 
    
    mean_order = np.mean([len(D) for D in Ds[n, p]])
    mean_diam = np.mean([D.max() for D in Ds[n, p]])
    for max_iter, rel_size_diff, c_seq in product(max_iters, rel_size_diffs, c_seqs):
        i += 1
        
        perf_key = n, p, max_iter, rel_size_diff, tuple(c_seq)
        if perf_key in performance:
            print(f'SKIPPING {i}/{n_runs}', compile_performance(*perf_key))
            continue
        
        start = time.time()
        exact_XY = []
        iter_seqs = [None]*N
        for j, (DX, P0s, pi_YX) in enumerate(zip(Ds[n, p], P0s_by_n_p[n, p], pi_YXs[n, p])):
            ix_ = np.ix_(pi_YX, pi_YX)
            if rel_size_diff == 0:
                DY = DX[ix_]
                # Make optimal mapping non-injective.
#                 AX_ = AX.copy()
#                 AX = np.zeros((len(AX_) + 1, len(AX_) + 1))
#                 AX[:len(AX_), :len(AX_)] = AX_
#                 AX[len(AX_), 0] = AX[0, len(AX_)] = .01
#                 DX = sps.csgraph.shortest_path(AX, directed=False)
#                 AY = AX.copy()
#                 AY[len(AX_), 0] = AY[0, len(AX_)] = 100
#                 DY = sps.csgraph.shortest_path(AY, directed=False)
#             else:
#                 size_diff = round(rel_size_diff * len(AX))
#                 AY_ = AX[np.ix_(pi_YX, pi_YX)]
#                 AY = np.zeros((len(AX) + size_diff, len(AX) + size_diff))
#                 AY[:len(AX), :len(AX)] = AY_
#                 AY[len(AX):, :] = AY[:, len(AX):] = DX.max() # to guarantee compliance with triangle ineq.
#                 AY[np.arange(size_diff) + len(AX), np.arange(size_diff) + len(AX)] = 0
#                 DY = sps.csgraph.shortest_path(AY, directed=False)
#                 assert np.all(DY[:len(AX), :len(AX)] == DX[np.ix_(pi_YX, pi_YX)]), 'extra points changed Y'
                            
            dis_ = partial(dis, DX=DX, DY=DY)
            ub_XY = np.inf
            fw_seq = [make_fw(DX, DY, c=c, dis_=dis_) for c in c_seq]
            
#             for P0 in P0s_by_n_p[n, p][j]:
            for P0 in [None]:#!!
                P_star, iter_seq = frank_wolfe_sequence(fw_seq, verbose=False, P0=P0, max_iter=max_iter)
                ub = dis_(project_P(P_star))/2
                if ub < ub_XY:
                    ub_XY = ub
                    iter_seqs[j] = iter_seq
                    
            exact_XY.append(ub_XY == 0)

        performance[perf_key] = {'acc': np.mean(exact_XY), 'time': (time.time() - start)/N,
                          'iters': np.mean(iter_seqs, axis=0), 'order': mean_order, 'diam': mean_diam}

        print(f'{i}/{n_runs}:', compile_performance(*perf_key))


SKIPPING 1/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.00032[3.23] → 2[20.09]: 10.0% correct, 0.027s
SKIPPING 2/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.0016[56.69] → 2[12.33]: 16.0% correct, 0.079s
SKIPPING 3/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.008[15.32] → 2[11.17]: 10.0% correct, 0.031s
SKIPPING 4/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.04[14.18] → 2[7.07]: 13.0% correct, 0.027s
SKIPPING 5/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.1[18.16] → 2[11.25]: 18.0% correct, 0.037s
SKIPPING 6/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.2[14.15] → 2[8.06]: 11.0% correct, 0.030s
SKIPPING 7/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.4[15.48] → 2[9.15]: 14.0% correct, 0.029s
SKIPPING 8/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=1.6[16.66] → 2[9.39]: 13.0% correct, 0.031s
SKIPPING 9/27 n=10 | p=0.25 | max#i=100 | #X=#Y | c[#i]=2[23.06]: 10.0% correct, 0.029s
SKIPPING 10/27 n=100 | p=0.05 | max#i=100 | #X=#Y | c[#i]=1.00032[98.54] → 2[15.18]: 20.0% correct,

In [2]:
ds = {False: [], True: []}
n = 1
for i in range(n):
    _, DX = gen_graph(1000, .01)
    pi = np.random.permutation(len(DX))
    DY = DX[np.ix_(pi, pi)]
    dis_ = partial(dis, DX=DX, DY=DY)
    
#    fw_seq = [make_fw(D2K(DX, t=.5), D2K(DY, t=.5), c=c, dis_=dis_) for c in [1.5]] + [make_fw(DX, DY, c=1.5, dis_=dis_)]
    fw_seq = [make_fw(DX, DY, c=1.5, dis_=dis_)]
    for is_away_step in [False, True][1:]:
        fw_desc = 'away step' if is_away_step else 'regular'
        P_star, iters = frank_wolfe_sequence(fw_seq, max_iter=100, is_away_step=is_away_step, verbose=1)
        ds[is_away_step].append(dis_(project_P(P_star)))
        print(f'{i}/{n}: dis={ds[is_away_step][-1]}, iters={iters}, {fw_desc} FW')

iter 1: dis(P)=3.213, dis(Proj P)=6.0, f(P)=2153420.94, (to) Q, α=0.03562 
iter 2: dis(P)=3.279, dis(Proj P)=5.0, f(P)=2152920.40, (to) Q, α=0.05748 
iter 3: dis(P)=3.356, dis(Proj P)=5.0, f(P)=2152124.93, (to) Q, α=0.06474 
iter 4: dis(P)=3.402, dis(Proj P)=5.0, f(P)=2151169.11, (to) Q, α=0.05173 
iter 5: dis(P)=3.412, dis(Proj P)=5.0, f(P)=2150365.39, (to) Q, α=0.07147 
iter 6: dis(P)=3.414, dis(Proj P)=5.0, f(P)=2149240.44, (to) Q, α=0.09987 
iter 7: dis(P)=3.484, dis(Proj P)=5.0, f(P)=2147643.43, (to) Q, α=0.05192 
iter 8: dis(P)=3.497, dis(Proj P)=5.0, f(P)=2146800.99, (to) Q, α=0.09852 
iter 9: dis(P)=3.547, dis(Proj P)=5.0, f(P)=2145270.43, (to) Q, α=0.10992 
iter 10: dis(P)=3.537, dis(Proj P)=5.0, f(P)=2143529.05, (to) Q, α=0.14531 
iter 11: dis(P)=3.501, dis(Proj P)=5.0, f(P)=2141279.06, (to) Q, α=0.08754 
iter 12: dis(P)=3.524, dis(Proj P)=5.0, f(P)=2139920.52, (to) Q, α=0.16516 
iter 13: dis(P)=3.468, dis(Proj P)=5.0, f(P)=2137545.42, (to) Q, α=0.19683 
iter 14: dis(P)=3.499

In [17]:
D1.shape

(100, 100)

In [4]:
for is_away_step in ds:
    success_rate = np.mean(np.array(ds[is_away_step]) == 0)
    print(f'accuracy: {success_rate:.2%}')

accuracy: 19.00%
accuracy: 19.00%


In [6]:
# WARNING: DELETION.
ks = performance.copy()
for n, p, max_iter, inj, size, obj_type, mx_type in ks:
    if n == 10:
        del performance[n, p, max_iter, inj, size, obj_type, mx_type]

In [52]:
#for n, p, max_iter, inj, size, obj_type, mx_type in product(ns, ps, max_iters, injs, sizes, obj_types, mx_types):
#    print(compile_performance(n, p, max_iter, inj, size, obj_type, mx_type))
for n, p, max_iter, inj, size, obj_type, mx_type in sorted(performance.keys()):
#    if ((obj_type == 'mix+' and inj == 0) or (obj_type == 'sq+' and inj == 1)) and mx_type == 1:
#    if size == 1:
    #if n == 100 and obj_type == 'sq':
        print(compile_performance(n, p, max_iter, inj, size, obj_type, mx_type, show_graph_properties=False))


n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(D): 48.0% correct, 3s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(K-exp) → c_exp(D): 46.0% correct, 26s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(K-exp_t) → c_exp(D): 46.0% correct, 25s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(K-exp) → exp(D): 50.0% correct, 21s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(K-exp_t) → exp(D): 48.0% correct, 21s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(K-exp) → sq+(D): 54.0% correct, 95s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | c_exp(K-exp_t) → sq+(D): 52.0% correct, 97s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | exp(D): 38.0% correct, 5s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | exp(K-exp) → c_exp(D): 54.0% correct, 24s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | exp(K-exp_t) → c_exp(D): 56.0% correct, 23s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | exp(K-exp) → exp(D): 48.0% correct, 18s
n=10 | p=0.25 | max_i=100 | non-inj | #X=#Y | exp(K-

In [ ]:
'''
n=100, p=0.05, max_iter=100, inj, X=Y, ‖Δ‖², AX & AY: 48.0% correct in 4s
n=100, p=0.05, max_iter=100, inj, X=Y, ‖Δ‖², DX & DY: 100.0% correct in 1s
n=100, p=0.05, max_iter=100, inj, X=Y, ‖e^|Δ|‖_1, AX & AY: 60.0% correct in 15s
n=100, p=0.05, max_iter=100, inj, X=Y, ‖e^|Δ|‖_1, DX & DY: 30.0% correct in 15s
n=500, p=0.05, max_iter=100, inj, X=Y, ‖Δ‖², AX & AY: 98.0% correct in 75s
n=500, p=0.05, max_iter=100, inj, X=Y, ‖Δ‖², DX & DY: 100.0% correct in 29s
n=500, p=0.05, max_iter=100, inj, X=Y, ‖e^|Δ|‖_1, AX & AY: 86.0% correct in 402s
n=500, p=0.05, max_iter=100, inj, X=Y, ‖e^|Δ|‖_1, DX & DY: 100.0% correct in 169s
n=10, p=0.1, max_iter=100, inj, X=Y, ‖Δ‖², AX & AY: 88.0% correct in 0s
n=10, p=0.1, max_iter=100, inj, X=Y, ‖Δ‖², DX & DY: 90.0% correct in 0s
n=10, p=0.1, max_iter=100, inj, X=Y, ‖e^|Δ|‖_1, AX & AY: 90.0% correct in 0s
n=10, p=0.1, max_iter=100, inj, X=Y, ‖e^|Δ|‖_1, DX & DY: 86.0% correct in 0s
'''

In [597]:
AX0, DX0, pi_YX0 = AX.copy(), DX.copy(), pi_YX.copy() # conserve problematic combination (AX,AY work but DX,DY don't on 'sq', inj=1)

In [1035]:
AX, DX, pi_YX = AX0.copy(), DX0.copy(), pi_YX0.copy() # restore the problematic combination
n = len(AX)
AY_ = AX[np.ix_(pi_YX, pi_YX)]
DY_ = DX[np.ix_(pi_YX, pi_YX)]
AY = np.zeros((n + 1, n + 1))
AY[:n, :n] = AY_
AY[n - 1, n] = AY[n, n - 1] = DX.max()
DY = sps.csgraph.shortest_path(AY, directed=False)
assert np.all(DY[:len(AX), :len(AX)] == DX[np.ix_(pi_YX, pi_YX)])

In [683]:
AX, DX = generate_graph(20, .1, enforce_n=True) # generate a combination
n = len(AX)
pi_YX = np.random.permutation(n)
AY_ = AX[np.ix_(pi_YX, pi_YX)]
DY_ = DX[np.ix_(pi_YX, pi_YX)]
AY = np.zeros((n + 1, n + 1))
AY[:n, :n] = AY_
AY[n - 1, n] = AY[n, n - 1] = DX.max()
DY = sps.csgraph.shortest_path(AY, directed=False)

In [1045]:
#find_ub_XY(DX, DY, partial(dis, DX=DX, DY=DY), 1, 100, 0, True)
#_,_,Pi = find_ub_XY(DY, DX, partial(dis, DX=DY, DY=DX), 1, 100, 1, True)
#dis(Pi[[0, 1, 2, 3, 4, 5, 6, 7]], DY, DX), Pi

inj_ub, ub, Pi_from_center = find_ub_XY(DX, DY, DX, DY, 'mix', 1000, 0, True)
# Pi_correct=[18,  0,  5, 14,  6, 17,  3, 11, 19,  7,  1, 12,  9, 10, 13, 15,  8, 2,  4, 16]
# pi_XY==Pi_correct for DX, DY_
inj_ub, ub, P2p(Pi_from_center)#, dis(Pi_XY, DX, DY)

----------INIT dis(P)=3.043083900226758, dis_l2(P)=718.2345524755629, dis(Pi)=6.0,dis_l2(Pi)=718.2345524755629, f(P)=-2844.5532879818593
iter 0: dis(P)=10.0, dis_l2(P)=1378.0, dis(Pi)=10.0,dis_l2(Pi)=1378.0, f(P)=-3550.0, α=[1.], ‖∇f‖²=41614243.99,‖α*dQP‖²=19.05 
iter 1: dis(P)=10.0, dis_l2(P)=1354.0, dis(Pi)=10.0,dis_l2(Pi)=1354.0, f(P)=-3562.0, α=[1.], ‖∇f‖²=46746432.00,‖α*dQP‖²=8.00 
iter 2: dis(P)=10.0, dis_l2(P)=1334.0, dis(Pi)=10.0,dis_l2(Pi)=1334.0, f(P)=-3572.0, α=[1.], ‖∇f‖²=46726980.00,‖α*dQP‖²=4.00 
----------STOP α=[0.5], ‖∇f‖²=46712668.00, ‖α*dQP‖²=0.00 
----------INIT dis(P)=10.0, dis_l2(P)=1334.0, dis(Pi)=10.0,dis_l2(Pi)=1334.0, f(P)=50993.59042134862
iter 0: dis(P)=4.9772435315624355, dis_l2(P)=1220.1681104719905, dis(Pi)=5.0,dis_l2(Pi)=1220.1681104719905, f(P)=3568.799159954821, α=[0.99544871], ‖∇f‖²=99617261804.35,‖α*dQP‖²=35.67 
iter 1: dis(P)=2.789423687223117, dis_l2(P)=466.8138121777503, dis(Pi)=3.0,dis_l2(Pi)=466.8138121777503, f(P)=1552.947271233746, α=[0.944192

(5.0,
 0.0,
 array([18,  0,  5, 14,  6, 17,  3, 11, 19,  7,  1, 12,  9, 10, 13, 15,  8,
         2,  4, 16]))

In [1027]:
Pi0 = p2P(np.argsort(pi_YX), len(DY)) # correct mapping X → Y (dis = 0)
Pi = Pi_from_center
P2p(Pi0)

array([18,  0,  5, 14,  6, 17,  3, 11, 19,  7,  1, 12,  9, 10, 13, 15,  8,
        2,  4, 16])

In [532]:
'''
[Pi0 = correct, Pi = obtained from F-W minimization (‖Δ‖², DX, DY, injections)] with no initialization

Why 'mix' recovered the isomorphism but 'mix+' did not?

'''


'\n!! why correct Pi_XY is not a local minimum for ‖Δ‖² on DX, DY and injections (but it is on AX, AY)?\nIs it because P.T@P != I (since |DY| > |DX|) and even for injective mapping:\nargmin ‖DX - P@DY@P.T‖² = argmin -2<P, DX@P@DY> + ‖P@DY@P.T‖² != argmin -<P, DX@P@DY>\n?\n'

In [1028]:
print('-<Pi, DX@Pi@DY>={}, -<Pi0, DX@Pi0@DY>={}, ‖Pi@DY@Pi.T‖²={}, ‖Pi0@DY@Pi0.T‖²={}'.format(
    np.sum(Pi * (-DX@Pi@DY)), np.sum(Pi0 * (-DX@Pi0@DY)),
    np.sum((Pi@DY@Pi.T)**2), np.sum((Pi0@DY@Pi0.T)**2)
))
print('-<Pi, AX@Pi@AY>={}, -<Pi0, AX@Pi0@AY>={}, ‖Pi@AY@Pi.T‖²={}, ‖Pi0@AY@Pi0.T‖²={}'.format(
    np.sum(Pi * (-AX@Pi@AY)), np.sum(Pi0 * (-AX@Pi0@AY)),
    np.sum((Pi@AY@Pi.T)**2), np.sum((Pi0@AY@Pi0.T)**2)
))
2*np.sum(Pi * (-DX@Pi@DY)) + np.sum((Pi@DY@Pi.T)**2), 2*np.sum(Pi0 * (-DX@Pi0@DY)) + np.sum((Pi0@DY@Pi0.T)**2)

-<Pi, DX@Pi@DY>=-2578.0, -<Pi0, DX@Pi0@DY>=-2910.0, ‖Pi@DY@Pi.T‖²=2766.0, ‖Pi0@DY@Pi0.T‖²=2910.0
-<Pi, AX@Pi@AY>=-26.0, -<Pi0, AX@Pi0@AY>=-66.0, ‖Pi@AY@Pi.T‖²=68.0, ‖Pi0@AY@Pi0.T‖²=66.0


(-2390.0, -2910.0)

In [1032]:
# Shows that minimization diverges from initially correct mapping (this time it's b/c of |Y| > |X|:
# argmin ‖DX - P@DY@P.T‖² = argmin -2<P, DX@P@DY> + ‖P@DY@P.T‖² != argmin -<P, DX@P@DY>).
# Using 'sq+' instead of 'sq' seems to fix it.
f, grad_f, alpha_jac, alpha_hess = def_functions_for_frank_wolfe(DX, DY, 'sq')
dis_ = partial(dis, DX=DX, DY=DY)
dis_l2_ = partial(dis_l2, DX=DX, DY=DY)
Pi_from_Pi0 = frank_wolfe(
    len(DX), len(DY), f, grad_f, alpha_jac, alpha_hess, dis_, dis_l2_,
    P0=Pi0,
    max_iter=max_iter, is_inj=True, verbose=True)
print(dis_(Pi_from_Pi0), dis_(Pi0), np.sum(Pi_from_Pi0 * (-DX@Pi_from_Pi0@DY)), np.sum(Pi0 * (-DX@Pi0@DY)))
2*np.sum(Pi_from_Pi0 * (-DX@Pi_from_Pi0@DY)) + np.sum((Pi_from_Pi0@DY@Pi_from_Pi0.T)**2), 2*np.sum(Pi0 * (-DX@Pi0@DY)) + np.sum((Pi0@DY@Pi0.T)**2)

----------INIT dis(P)=3.0, dis_l2(P)=520.0, dis(Pi)=3.0,dis_l2(Pi)=520.0, f(P)=-1195.0
iter 0: dis(P)=2.780997591989085, dis_l2(P)=225.07424437484195, dis(Pi)=3.0,dis_l2(Pi)=225.07424437484195, f(P)=-1342.4628778125789, α=[0.27337848], ‖∇f‖²=1826084.00,‖α*dQP‖²=2.99 
iter 1: dis(P)=2.6503266505911114, dis_l2(P)=193.2216292859594, dis(Pi)=3.0,dis_l2(Pi)=193.2216292859594, f(P)=-1358.3891853570203, α=[0.122432], ‖∇f‖²=401259.97,‖α*dQP‖²=0.34 
iter 2: dis(P)=3.064566937643172, dis_l2(P)=182.22661239002818, dis(Pi)=3.0,dis_l2(Pi)=182.22661239002818, f(P)=-1363.886693804986, α=[0.09501403], ‖∇f‖²=306284.01,‖α*dQP‖²=0.21 
iter 3: dis(P)=3.030833810194699, dis_l2(P)=180.99947474069558, dis(Pi)=3.0,dis_l2(Pi)=180.99947474069558, f(P)=-1364.500262629652, α=[0.02639734], ‖∇f‖²=356299.30,‖α*dQP‖²=0.01 
iter 4: dis(P)=2.987270889014601, dis_l2(P)=179.5219953166143, dis(Pi)=3.0,dis_l2(Pi)=179.5219953166143, f(P)=-1365.2390023416929, α=[0.03232721], ‖∇f‖²=364730.70,‖α*dQP‖²=0.02 
iter 5: dis(P)=3.12

iter 67: dis(P)=3.008538101476587, dis_l2(P)=157.03509074883632, dis(Pi)=4.0,dis_l2(Pi)=157.03509074883632, f(P)=-1376.4824546255818, α=[0.02010532], ‖∇f‖²=341812.10,‖α*dQP‖²=0.00 
iter 68: dis(P)=3.0387113693424364, dis_l2(P)=156.9601085315553, dis(Pi)=4.0,dis_l2(Pi)=156.9601085315553, f(P)=-1376.5199457342223, α=[0.00889952], ‖∇f‖²=329391.77,‖α*dQP‖²=0.00 
iter 69: dis(P)=3.027615142012515, dis_l2(P)=156.88284151837135, dis(Pi)=4.0,dis_l2(Pi)=156.88284151837135, f(P)=-1376.5585792408142, α=[0.00754799], ‖∇f‖²=336144.38,‖α*dQP‖²=0.00 
iter 70: dis(P)=3.0018022547669214, dis_l2(P)=156.77206198848478, dis(Pi)=4.0,dis_l2(Pi)=156.77206198848478, f(P)=-1376.6139690057576, α=[0.0172354], ‖∇f‖²=338863.03,‖α*dQP‖²=0.00 
iter 71: dis(P)=3.0403196422215277, dis_l2(P)=156.6519117770475, dis(Pi)=4.0,dis_l2(Pi)=156.6519117770475, f(P)=-1376.6740441114762, α=[0.0112695], ‖∇f‖²=330230.12,‖α*dQP‖²=0.00 
iter 72: dis(P)=3.0321301038949886, dis_l2(P)=156.60939770628377, dis(Pi)=4.0,dis_l2(Pi)=156.60939

(-2380.0, -2910.0)

In [581]:
P2p(Pi), P2p(Pi_XY)

(array([18,  0,  5, 14,  6, 17,  3, 11, 20, 13,  1, 12,  9, 10, 19, 15,  8,
         2,  4, 16]),
 array([20,  0,  5, 14,  6, 17,  3, 11, 18, 19,  1, 12,  9, 10, 13, 15,  8,
         2,  4, 16]))

In [662]:
f, grad_f, alpha_jac, alpha_hess = def_functions_for_frank_wolfe(DX, DY, 'sq')
dis_ = partial(dis, DX=DX, DY=DY)
Pi = frank_wolfe(
    len(DX), len(DY), f, grad_f, alpha_jac, alpha_hess, dis_,
    P0=Pi_from_center,
    max_iter=max_iter, is_inj=True, verbose=True)
print(dis_(Pi), dis_(Pi0), np.sum(Pi * (-DX@Pi@DY)), np.sum(Pi0 * (-DX@Pi0@DY)))

----------INIT dis(P)=4.10, dis(Pi)=4.1, f(P)=-2870.2
----------STOP α=[0.5], ‖∇f‖²=26643304.84, ‖α*dQP‖²=0.00 
4.1 0.0 -2870.2 -2910.0
